In [134]:
import pandas as pd
import sqlite3
import os


In [135]:
#Checking column names and looking for join potential
df_NFL_Draft_2025 = pd.read_csv('NFL_Draft_2025.csv')
print(df_NFL_Draft_2025.columns.tolist())


['CollegeAthleteId', 'NflAthleteId', 'CollegeId', 'CollegeTeam', 'CollegeConference', 'NflTeamId', 'NflTeam', 'Year', 'Overall', 'Round', 'Pick', 'Name', 'Position', 'Height', 'Weight', 'PreDraftRanking', 'PreDraftPositionRanking', 'PreDraftGrade', 'HometownInfo City', 'HometownInfo State', 'HometownInfo Country', 'HometownInfo Latitude', 'HometownInfo Longitude', 'HometownInfo CountyFips']


In [136]:
# Checking column names and looking for join potential
df_Recruiting_Ranking_2022 = pd.read_csv('Recruiting_Ranking_2022.csv')
print(df_Recruiting_Ranking_2022.columns)

Index(['Id', 'AthleteId', 'RecruitType', 'Year', 'Ranking', 'Name', 'School',
       'CommittedTo', 'Position', 'Height', 'Weight', 'Stars', 'Rating',
       'City', 'StateProvince', 'Country', 'HometownInfo Latitude',
       'HometownInfo Longitude', 'HometownInfo FipsCode'],
      dtype='object')


In [137]:
#Combining Highschool recruiting ranking csv for years 202019-2023 ( there should be 4 people per rank)
# Added more years to appease a friend and get specific data to compare

files_to_combine = [
    'Recruiting_Ranking_2015.csv',
    'Recruiting_Ranking_2016.csv',
    'Recruiting_Ranking_2017.csv',
    'Recruiting_Ranking_2018.csv',
    'Recruiting_Ranking_2019.csv',
    'Recruiting_Ranking_2020.csv',
    'Recruiting_Ranking_2021.csv',
    'Recruiting_Ranking_2022.csv',
    'Recruiting_Ranking_2019.csv',
]

Recruiting_Ranking = []

for file in files_to_combine:
    College = pd.read_csv(file)
    Recruiting_Ranking.append(College)

Recruiting_Rank_Combined = pd.concat(Recruiting_Ranking, ignore_index=True)

print(Recruiting_Rank_Combined.head(10))

      Id  AthleteId RecruitType  Year  Ranking              Name  \
0  66928  3915192.0  HighSchool  2015      1.0  Trenton Thompson   
1  31860        NaN  HighSchool  2015      1.0    Trent Thompson   
2  31861 -1009710.0  HighSchool  2015      2.0       Martez Ivey   
3  31862  3916922.0  HighSchool  2015      3.0      Byron Cowart   
4  31863  3912545.0  HighSchool  2015      4.0     Iman Marshall   
5  31864  3691739.0  HighSchool  2015      5.0      Derwin James   
6  31865  4266441.0  HighSchool  2015      6.0   Kahlil McKenzie   
7  31866        NaN  HighSchool  2015      7.0    CeCe Jefferson   
8  31867  3693166.0  HighSchool  2015      8.0        Josh Sweat   
9  31868  3843234.0  HighSchool  2015      9.0  Kevin Toliver II   

                      School    CommittedTo Position  Height  Weight  Stars  \
0                   Westover        Georgia       DT    74.0   313.0      5   
1                   Westover        Georgia       DT    74.5   313.0      5   
2             

In [138]:
# Combining NFL Draft results from 2022-2025
# Added more draft years to appease a friend and compare specific data
files_to_combine_draft = [
    'NFL_Draft_2019.csv',
    'NFL_Draft_2020.csv',
    'NFL_Draft_2021.csv',
    'NFL_Draft_2022.csv',
    'NFL_Draft_2023.csv',
    'NFL_Draft_2024.csv',
    'NFL_Draft_2025.csv'
]
NFL_Draft = []
for file in files_to_combine_draft:
    NFL = pd.read_csv(file)
    NFL_Draft.append(NFL)

NFL_Draft_Combined = pd.concat(NFL_Draft, ignore_index=True)

print(NFL_Draft_Combined.head(10))

   CollegeAthleteId  NflAthleteId  CollegeId CollegeTeam  CollegeConference  \
0         3917315.0        103832        201    Oklahoma             Big 12   
1         4040605.0        103504        194  Ohio State            Big Ten   
2         4040982.0        103509        333     Alabama                SEC   
3         3728258.0        103514        228     Clemson                ACC   
4         4035434.0        103507         99         LSU                SEC   
5         3917792.0        103572        150        Duke                ACC   
6         3915239.0        103524         96    Kentucky                SEC   
7         4036133.0        103545       2294        Iowa            Big Ten   
8         4039303.0        103505        248     Houston  American Athletic   
9         4036261.0        103522        130    Michigan            Big Ten   

   NflTeamId        NflTeam  Year  Overall  Round  ...  Weight  \
0         22        Arizona  2019        1      1  ...   207.0  

In [139]:
# Looking at results
print(NFL_Draft_Combined.columns)

Index(['CollegeAthleteId', 'NflAthleteId', 'CollegeId', 'CollegeTeam',
       'CollegeConference', 'NflTeamId', 'NflTeam', 'Year', 'Overall', 'Round',
       'Pick', 'Name', 'Position', 'Height', 'Weight', 'PreDraftRanking',
       'PreDraftPositionRanking', 'PreDraftGrade', 'HometownInfo City',
       'HometownInfo State', 'HometownInfo Country', 'HometownInfo Latitude',
       'HometownInfo Longitude', 'HometownInfo CountyFips'],
      dtype='object')


In [140]:
# looking at results
print(NFL_Draft_Combined.head(15))

    CollegeAthleteId  NflAthleteId  CollegeId     CollegeTeam  \
0          3917315.0        103832        201        Oklahoma   
1          4040605.0        103504        194      Ohio State   
2          4040982.0        103509        333         Alabama   
3          3728258.0        103514        228         Clemson   
4          4035434.0        103507         99             LSU   
5          3917792.0        103572        150            Duke   
6          3915239.0        103524         96        Kentucky   
7          4036133.0        103545       2294            Iowa   
8          4039303.0        103505        248         Houston   
9          4036261.0        103522        130        Michigan   
10         4040726.0        103513        333         Alabama   
11         4046523.0        103508        130        Michigan   
12         3728266.0        103516        228         Clemson   
13         3672833.0        103621        103  Boston College   
14         4040616.0     

In [141]:
#Looking at shape to see if requirements were met
NFL_Draft_Combined.shape

(1803, 24)

In [142]:
Recruiting_Rank_Combined.columns

Index(['Id', 'AthleteId', 'RecruitType', 'Year', 'Ranking', 'Name', 'School',
       'CommittedTo', 'Position', 'Height', 'Weight', 'Stars', 'Rating',
       'City', 'StateProvince', 'Country', 'HometownInfo Latitude',
       'HometownInfo Longitude', 'HometownInfo FipsCode'],
      dtype='object')

In [143]:
# Renaming columns for consistency as this will be my key in SQL
Recruiting_Rank_Combined.rename(columns={'AthleteId': 'CollegeAthleteId'}, inplace=True)

In [144]:
Recruiting_Rank_Combined.columns

Index(['Id', 'CollegeAthleteId', 'RecruitType', 'Year', 'Ranking', 'Name',
       'School', 'CommittedTo', 'Position', 'Height', 'Weight', 'Stars',
       'Rating', 'City', 'StateProvince', 'Country', 'HometownInfo Latitude',
       'HometownInfo Longitude', 'HometownInfo FipsCode'],
      dtype='object')

In [145]:
#deleted columns 
Recruiting_Rank_Combined.drop(['HometownInfo FipsCode', 'RecruitType'], axis=1, inplace=True)



In [146]:
#chekcing my work to make sure the columns were deleted
Recruiting_Rank_Combined.columns

Index(['Id', 'CollegeAthleteId', 'Year', 'Ranking', 'Name', 'School',
       'CommittedTo', 'Position', 'Height', 'Weight', 'Stars', 'Rating',
       'City', 'StateProvince', 'Country', 'HometownInfo Latitude',
       'HometownInfo Longitude'],
      dtype='object')

In [147]:
Recruiting_Rank_Combined.shape

(33609, 17)

In [148]:
Recruiting_Rank_Combined.isnull().sum()
print(Recruiting_Rank_Combined.isnull().sum())

Id                            0
CollegeAthleteId          11713
Year                          0
Ranking                     399
Name                          0
School                      275
CommittedTo                6743
Position                     19
Height                       64
Weight                       68
Stars                         0
Rating                        0
City                        338
StateProvince               341
Country                     216
HometownInfo Latitude       470
HometownInfo Longitude      470
dtype: int64


In [149]:
Recruiting_Rank_Combined[Recruiting_Rank_Combined["CommittedTo"].isnull()]

,Id,CollegeAthleteId,Year,Ranking,Name,School,CommittedTo,Position,Height,Weight,Stars,Rating,City,StateProvince,Country,HometownInfo Latitude,HometownInfo Longitude
290,66934,NaN,2015,285.0,Ahmad Wagner,Wayne,NaN,WR,79.0,190.0,4,0.9000,Dayton,OH,USA,39.758948,-84.191607
310,32154,NaN,2015,303.0,Natereace Strong,East St. Louis,NaN,RB,73.0,210.0,4,0.8976,East Saint Louis,IL,USA,38.624495,-90.150943
342,32182,4242246.0,2015,332.0,Tommy Champion,Callaway,NaN,OT,77.0,285.0,4,0.8913,Jackson,MS,USA,32.299038,-90.184769
418,32316,NaN,2015,408.0,Jaylon Lane,Nacogdoches,NaN,CB,73.0,177.0,3,0.8789,Nacogdoches,TX,USA,31.603513,-94.655487
570,32412,3932502.0,2015,557.0,Brian Bell,Lowndes,NaN,OLB,74.0,218.0,3,0.8650,Valdosta,GA,USA,30.832702,-83.278485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33598,47923,NaN,2019,NaN,Donivan Wright,Lawrence Central,NaN,WR,72.0,165.0,3,0.8499,Indianapolis,IN,USA,39.768333,-86.158350
33600,50880,4570068.0,2019,NaN,Cooper Nelson,Sun Prairie,NaN,WR,76.0,180.0,1,0.6996,Sun Prairie,WI,USA,43.183458,-89.213436
33601,48524,4569648.0,2019,NaN,Zion Sorani,Cathedral Catholic,NaN,WR,72.0,170.0,3,0.8299,San Diego,CA,USA,32.717421,-117.162771
33603,48241,NaN,2019,NaN,David Heckard,Faith Lutheran,NaN,WDE,77.0,235.0,3,0.8376,Las Vegas,NV,USA,36.167256,-115.148516


In [150]:
Recruiting_Rank_Combined[Recruiting_Rank_Combined["Name"] == "Zach Evans"]

,Id,CollegeAthleteId,Year,Ranking,Name,School,CommittedTo,Position,Height,Weight,Stars,Rating,City,StateProvince,Country,HometownInfo Latitude,HometownInfo Longitude
20151,74906,4685035.0,2020,16.0,Zach Evans,North Shore,TCU,RB,71.0,200.0,5,0.9925,Houston,TX,USA,29.758938,-95.367697
27868,90773,4702901.0,2022,670.0,Zach Evans,Rockwall-Heath,Minnesota,RB,69.0,200.0,3,0.8748,Rockwall,TX,USA,32.931234,-96.459709


In [151]:
NFL_Draft_Combined[NFL_Draft_Combined["Name"] == "Zach Evans"]

,CollegeAthleteId,NflAthleteId,CollegeId,CollegeTeam,CollegeConference,NflTeamId,NflTeam,Year,Overall,Round,...,Weight,PreDraftRanking,PreDraftPositionRanking,PreDraftGrade,HometownInfo City,HometownInfo State,HometownInfo Country,HometownInfo Latitude,HometownInfo Longitude,HometownInfo CountyFips
1244,4685035.0,106360,145,Ole Miss,SEC,14,Los Angeles,2023,215,6,...,202.0,160.0,13.0,56.0,Houston,TX,USA,29.758938,-95.367697,48201.0


In [152]:
#checking for null values in CollegeAthleteId
Recruiting_Rank_Combined[Recruiting_Rank_Combined["CollegeAthleteId"].isnull()]

,Id,CollegeAthleteId,Year,Ranking,Name,School,CommittedTo,Position,Height,Weight,Stars,Rating,City,StateProvince,Country,HometownInfo Latitude,HometownInfo Longitude
1,31860,NaN,2015,1.0,Trent Thompson,Westover,Georgia,DT,74.5,313.0,5,0.9991,Albany,GA,USA,31.578206,-84.155681
7,31866,NaN,2015,7.0,CeCe Jefferson,Baker County Senior,Florida,SDE,74.0,275.0,5,0.9951,Glen Saint Mary,FL,USA,30.275791,-82.160669
72,31929,NaN,2015,70.0,Trent Irwin,Hart,Stanford,WR,73.5,189.0,4,0.9629,Newhall,CA,USA,37.348542,-121.920399
104,31961,NaN,2015,102.0,Chidi Valentine-Okeke,Faith Baptist Christian Academy,LSU,OT,78.0,305.0,4,0.9507,Ludowici,GA,USA,31.709796,-81.743085
119,31975,NaN,2015,116.0,Jeffery Holland,Trinity Christian Academy,Auburn,OLB,74.0,230.0,4,0.9443,Jacksonville,FL,USA,30.332184,-81.655651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33597,48677,NaN,2019,NaN,Dajon Terry,Meridian,Kansas,DT,76.0,320.0,3,0.8227,Meridian,MS,USA,32.364310,-88.703656
33598,47923,NaN,2019,NaN,Donivan Wright,Lawrence Central,NaN,WR,72.0,165.0,3,0.8499,Indianapolis,IN,USA,39.768333,-86.158350
33603,48241,NaN,2019,NaN,David Heckard,Faith Lutheran,NaN,WDE,77.0,235.0,3,0.8376,Las Vegas,NV,USA,36.167256,-115.148516
33605,49519,NaN,2019,NaN,Joseph Goins Jr.,National Christian Academy,NaN,RB,70.0,210.0,2,0.7953,Fort Washington,MD,USA,38.712530,-77.012460


In [153]:
NFL_Draft_Combined[NFL_Draft_Combined["CollegeAthleteId"].isnull()]

,CollegeAthleteId,NflAthleteId,CollegeId,CollegeTeam,CollegeConference,NflTeamId,NflTeam,Year,Overall,Round,...,Weight,PreDraftRanking,PreDraftPositionRanking,PreDraftGrade,HometownInfo City,HometownInfo State,HometownInfo Country,HometownInfo Latitude,HometownInfo Longitude,HometownInfo CountyFips
676,NaN,105422,1000928,Central Missouri,NaN,16,Minnesota,2021,168,5,...,245.0,179.0,8.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN
1520,NaN,108744,2837,East Texas A&M,Southland,16,Minnesota,2024,232,7,...,269.0,388.0,32.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
#Creating SQLite database and saving the combined Recruiting Ranking dataframe
conn= sqlite3.connect('Football.db')

Recruiting_Rank_Combined.to_sql("Recruiting_Ranking_Combined", conn, if_exists='replace', index=False)
NFL_Draft_Combined.to_sql("NFL_Draft_Combined", conn, if_exists='replace', index=False)

1803

In [155]:
# Function to create a SQLite connection (1)
def sql_conn():
    return sqlite3.connect('Football.db')

In [156]:
with sql_conn() as conn:
    df = pd.read_sql_query("""
        SELECT d.CollegeAthleteId
        FROM Recruiting_Ranking_Combined r
        RIGHT JOIN NFL_Draft_Combined d
        ON d.CollegeAthleteId = r.CollegeAthleteId;
    """, conn)

df.columns


Index(['CollegeAthleteId'], dtype='object')

In [157]:
# Added latitude and longitude to the change in rating dataframe and realized I needed to change the column names to avoid issues with spaces

NFL_Draft_Combined= NFL_Draft_Combined.rename(columns={
    'HometownInfo Latitude': 'HometownInfoLatitude',
    'HometownInfo Longitude': 'HometownInfoLongitude'
})
NFL_Draft_Combined.columns

Index(['CollegeAthleteId', 'NflAthleteId', 'CollegeId', 'CollegeTeam',
       'CollegeConference', 'NflTeamId', 'NflTeam', 'Year', 'Overall', 'Round',
       'Pick', 'Name', 'Position', 'Height', 'Weight', 'PreDraftRanking',
       'PreDraftPositionRanking', 'PreDraftGrade', 'HometownInfo City',
       'HometownInfo State', 'HometownInfo Country', 'HometownInfoLatitude',
       'HometownInfoLongitude', 'HometownInfo CountyFips'],
      dtype='object')

In [158]:
    # There is something wrong here. I out of 1,000 lines that just over 50% can be joined 

with sql_conn() as conn:
    change_in_rating_df = pd.read_sql_query("""
        SELECT d.CollegeAthleteId, 
               d.Name,
               d.Position,
               d.CollegeTeam AS College, 
               r.Rating AS HighSchoolRank,
               d.NflTeam,
               d.Year AS DraftYear,
               d.Overall as Overall_Pick, 
               d.PreDraftGrade AS NFLPreDraftGrade,
               (d.PreDraftGrade / 100 - r.Rating) AS ChangeInRating,
               d."HometownInfo Latitude" AS HometownInfoLatitude,
               d."HometownInfo Longitude" AS HometownInfoLongitude
        FROM NFL_Draft_Combined d 
        LEFT JOIN Recruiting_Ranking_Combined r
        ON d.CollegeAthleteId = r.CollegeAthleteId;
    """, conn)

print(change_in_rating_df.head(15))
print(change_in_rating_df.count)



    CollegeAthleteId               Name            Position         College  \
0          3917315.0       Kyler Murray         Quarterback        Oklahoma   
1          4040605.0          Nick Bosa       Defensive End      Ohio State   
2          4040982.0   Quinnen Williams    Defensive Tackle         Alabama   
3          3728258.0     Clelin Ferrell       Defensive End         Clemson   
4          4035434.0        Devin White   Inside Linebacker             LSU   
5          3917792.0       Daniel Jones         Quarterback            Duke   
6          3915239.0         Josh Allen  Outside Linebacker        Kentucky   
7          4036133.0     T.J. Hockenson           Tight End            Iowa   
8          4039303.0          Ed Oliver    Defensive Tackle         Houston   
9          4036261.0         Devin Bush   Inside Linebacker        Michigan   
10         4040726.0     Jonah Williams    Offensive Tackle         Alabama   
11         4046523.0        Rashan Gary       Defens

In [159]:

Recruiting_Rank_Combined.shape


(33609, 17)

In [160]:
    # Checking for null vallues in NFL draft key column
with sql_conn() as conn:
    null_df = pd.read_sql_query("""
        SELECT CollegeAthleteId
        FROM NFL_Draft_Combined n
        WHERE n.CollegeAthleteId IS NULL;
    """, conn)

print(null_df)

  CollegeAthleteId
0             None
1             None


In [161]:
# Changing column PreDraftGrade to be a percentage for finding the change in rating from high school to draft 
    #Note to self - College rating will be 4 decimal places, NFL predraft rating will be 2 decimal places

NFL_Draft_Combined['PreDraftGrade']= NFL_Draft_Combined['PreDraftGrade']/100

print(NFL_Draft_Combined.head())

   CollegeAthleteId  NflAthleteId  CollegeId CollegeTeam CollegeConference  \
0         3917315.0        103832        201    Oklahoma            Big 12   
1         4040605.0        103504        194  Ohio State           Big Ten   
2         4040982.0        103509        333     Alabama               SEC   
3         3728258.0        103514        228     Clemson               ACC   
4         4035434.0        103507         99         LSU               SEC   

   NflTeamId        NflTeam  Year  Overall  Round  ...  Weight  \
0         22        Arizona  2019        1      1  ...   207.0   
1         25  San Francisco  2019        2      1  ...   266.0   
2         20       New York  2019        3      1  ...   303.0   
3         13      Las Vegas  2019        4      1  ...   264.0   
4         27      Tampa Bay  2019        5      1  ...   237.0   

  PreDraftRanking PreDraftPositionRanking  PreDraftGrade  HometownInfo City  \
0             9.0                     1.0           0.9

In [162]:
# Creating a new df with new column for the change in rating from high school to draft
    # Going to see if I can us this new df to see which postions have the most change in rating,

with sql_conn() as conn:
    change_in_rating_df = pd.read_sql_query("""
        SELECT d.CollegeAthleteId, 
               d.Name,
               d.Position,
               d.CollegeTeam AS College, 
               r.Rating AS HighSchoolRank,
               d.NflTeam,
               d.Year AS DraftYear,
               d.[HometownInfo Latitude] AS HometownInfoLatitude,
               d.[HometownInfo Longitude] AS HometownInfoLongitude,
               d.Overall as Overall_Pick, 
               d.PreDraftGrade AS NFLPreDraftGrade,
               (d.PreDraftGrade / 100 - r.Rating) AS ChangeInRating             
        FROM NFL_Draft_Combined d 
        LEFT JOIN Recruiting_Ranking_Combined r
        ON d.CollegeAthleteId = r.CollegeAthleteId;
    """, conn)

print(change_in_rating_df.head(15))

    CollegeAthleteId               Name            Position         College  \
0          3917315.0       Kyler Murray         Quarterback        Oklahoma   
1          4040605.0          Nick Bosa       Defensive End      Ohio State   
2          4040982.0   Quinnen Williams    Defensive Tackle         Alabama   
3          3728258.0     Clelin Ferrell       Defensive End         Clemson   
4          4035434.0        Devin White   Inside Linebacker             LSU   
5          3917792.0       Daniel Jones         Quarterback            Duke   
6          3915239.0         Josh Allen  Outside Linebacker        Kentucky   
7          4036133.0     T.J. Hockenson           Tight End            Iowa   
8          4039303.0          Ed Oliver    Defensive Tackle         Houston   
9          4036261.0         Devin Bush   Inside Linebacker        Michigan   
10         4040726.0     Jonah Williams    Offensive Tackle         Alabama   
11         4046523.0        Rashan Gary       Defens

In [163]:
change_in_rating_df.drop_duplicates(subset=['CollegeAthleteId'], inplace=True)

In [164]:
# Saving the  to a new table in the SQLite Table
with sql_conn() as conn:
    change_in_rating_df.to_sql("Change_In_Rating", conn, if_exists='replace', index=False)

In [165]:
with sql_conn() as conn:
    df = pd.read_sql_query("""
        SELECT *
        FROM Change_In_Rating
        LIMIT 15;
    """, conn)

print(df)

    CollegeAthleteId               Name            Position         College  \
0          3917315.0       Kyler Murray         Quarterback        Oklahoma   
1          4040605.0          Nick Bosa       Defensive End      Ohio State   
2          4040982.0   Quinnen Williams    Defensive Tackle         Alabama   
3          3728258.0     Clelin Ferrell       Defensive End         Clemson   
4          4035434.0        Devin White   Inside Linebacker             LSU   
5          3917792.0       Daniel Jones         Quarterback            Duke   
6          3915239.0         Josh Allen  Outside Linebacker        Kentucky   
7          4036133.0     T.J. Hockenson           Tight End            Iowa   
8          4039303.0          Ed Oliver    Defensive Tackle         Houston   
9          4036261.0         Devin Bush   Inside Linebacker        Michigan   
10         4040726.0     Jonah Williams    Offensive Tackle         Alabama   
11         4046523.0        Rashan Gary       Defens

In [166]:
#Look at how many rows I have in the Change_In_Rating table as it should be the same as NFL Draft_Combined

with sql_conn() as conn:
    count_df = pd.read_sql_query("""
        SELECT Count(*) 
        FROM Change_In_Rating;
    """, conn)
print(count_df)

   Count(*)
0      1802


In [167]:
print(change_in_rating_df)

      CollegeAthleteId              Name           Position       College  \
0            3917315.0      Kyler Murray        Quarterback      Oklahoma   
1            4040605.0         Nick Bosa      Defensive End    Ohio State   
2            4040982.0  Quinnen Williams   Defensive Tackle       Alabama   
3            3728258.0    Clelin Ferrell      Defensive End       Clemson   
4            4035434.0       Devin White  Inside Linebacker           LSU   
...                ...               ...                ...           ...   
2039         4683276.0      Zeek Biggers   Defensive Tackle  Georgia Tech   
2040         4429073.0       Fadil Diggs     Defensive Edge      Syracuse   
2041         4432338.0       Luke Lachey          Tight End          Iowa   
2042         4426513.0  Trikweze Bridges             Safety       Florida   
2044         4566100.0       Kobee Minor         Cornerback       Memphis   

      HighSchoolRank        NflTeam  DraftYear  HometownInfoLatitude  \
0  

In [168]:
#Creating a new file from Football.db because I found out that I cannot directly load a .db into my Tableau version

with sql_conn() as conn:
    Change_in_Rating= pd.read_sql_query("""
        SELECT *
        FROM Change_In_Rating """, conn)

Change_in_Rating.to_csv('Rating_Change.csv', index=False)

In [169]:
with sql_conn() as conn:
    Recruiting_Rank_Combined_Csv = pd.read_sql_query("""
        SELECT *
        FROM Recruiting_Ranking_Combined
        """, conn)
Recruiting_Rank_Combined_Csv.to_csv('Recruiting_Rank_Combined.csv', index=False)

In [170]:

with sql_conn() as conn:
    NFL_Draft= pd.read_sql_query("""
        SELECT *
        FROM NFL_Draft_Combined
        """, conn)
NFL_Draft.to_csv('NFL_Draft.csv', index=False)

In [171]:
# Making a function to return the information on a specific player that was drafted (2)

def get_player_info(Name):
    player_info_query = f"""
        SELECT *
        FROM Change_In_Rating
        WHERE LOWER(TRIM(Name)) 
        LIKE LOWER(?)
    """
    params= f"%{Name.strip()}%"

    with sql_conn() as conn:
        return pd.read_sql_query(player_info_query, conn, params=(params,))

In [172]:
#Checking accuracy of function

get_player_info('hardman')

,CollegeAthleteId,Name,Position,College,HighSchoolRank,NflTeam,DraftYear,HometownInfoLatitude,HometownInfoLongitude,Overall_Pick,NFLPreDraftGrade,ChangeInRating
0,4035004.0,Mecole Hardman,Wide Receiver,Georgia,0.9902,Kansas City,2019,34.20483,-83.0307,56,78.0,-0.2102


In [173]:
# Need to add 2018 and possibly 2017 Recruiting data to fill in the gaps for the NFL Draft data
# Unltimitely added 2015-2018 Recruiting data and 2019-2021 Draft data after speaking with a friend who wanted to see the data on a specific player

In [174]:
# Creating a function to return who each NFL team drated (function 3)

def get_NflTeam_Draft_results(NflTeam, DraftYear):
    team_draft_info_query = f"""
        SELECT *
        FROM Change_IN_Rating
        WHERE LOWER(TRIM(NflTeam))
        LIKE LOWER(?)
        """
    params = [f"%{NflTeam.strip()}%"]

    if DraftYear:
        team_draft_info_query += " AND DraftYear = ?"
        params.append(DraftYear)

    with sql_conn() as conn:
        return pd.read_sql_query(team_draft_info_query, conn, params=params)

In [175]:
get_NflTeam_Draft_results('New Orleans', 2023
 )

,CollegeAthleteId,Name,Position,College,HighSchoolRank,NflTeam,DraftYear,HometownInfoLatitude,HometownInfoLongitude,Overall_Pick,NFLPreDraftGrade,ChangeInRating
0,4428988.0,Bryan Bresee,Defensive Tackle,Clemson,0.9995,New Orleans,2023,39.288438,-77.203872,29,85.0,-0.1495
1,4426457.0,Isaiah Foskey,Defensive End,Notre Dame,0.9164,New Orleans,2023,38.004921,-121.805789,40,75.0,-0.1664
2,4599739.0,Kendre Miller,Running Back,TCU,0.8344,New Orleans,2023,31.917947,-94.681044,71,77.0,-0.0644
3,4361600.0,Nick Saldiveri,Offensive Tackle,Old Dominion,0.7798,New Orleans,2023,34.936253,-80.750792,103,66.0,-0.1198
4,4243322.0,Jake Haener,Quarterback,Fresno State,0.8599,New Orleans,2023,37.821593,-121.999961,127,71.0,-0.1499
5,4360949.0,Jordan Howden,Safety,Minnesota,NaN,New Orleans,2023,36.167256,-115.148516,146,52.0,NaN
6,4362019.0,A.T. Perry,Wide Receiver,Wake Forest,NaN,New Orleans,2023,26.615891,-80.069143,195,68.0,NaN


In [176]:
# Creating a function to search by College to see how many players were drafted from a specific college (function 4)

def get_college_info(College,DraftYear):
    college_info_query = f"""
        SELECT *
        FROM Change_IN_Rating
        WHERE LOWER(TRIM(College))
        LIKE LOWER(?)
        """
    params = [f"%{College.strip()}%"]

    if DraftYear:
        college_info_query += " AND DraftYear = ?"
        params.append(DraftYear)

    with sql_conn() as conn:
        return pd.read_sql_query(college_info_query, conn, params=params)

In [177]:
get_college_info('Lsu', 2023)

,CollegeAthleteId,Name,Position,College,HighSchoolRank,NflTeam,DraftYear,HometownInfoLatitude,HometownInfoLongitude,Overall_Pick,NFLPreDraftGrade,ChangeInRating
0,4429111.0,BJ Ojulari,Outside Linebacker,LSU,0.9634,Arizona,2023,33.952847,-84.549615,41,83.0,-0.1334
1,4429211.0,Anthony Bradford,Offensive Guard,LSU,0.9179,Seattle,2023,43.234181,-86.248392,108,52.0,-0.3979
2,4568007.0,Jay Ward,Safety,LSU,0.8801,Minnesota,2023,31.179998,-83.788793,134,58.0,-0.3001
3,4429034.0,Jaquelin Roy,Defensive Tackle,LSU,0.9803,Minnesota,2023,30.445960,-91.187380,141,62.0,-0.3603
4,4429022.0,Kayshon Boutte,Wide Receiver,LSU,0.9894,New England,2023,30.003536,-91.818729,187,51.0,-0.4794
5,4361834.0,Jarrick Bernard-Converse,Cornerback,LSU,0.8493,New York,2023,32.522183,-93.765194,204,32.0,-0.5293


In [178]:
# Get draft results by year (function 5)

def get_draft_results_by_year(DraftYear):
    draft_year_query = """
        SELECT *
        FROM Change_IN_Rating
        WHERE DraftYear = ?
    """
    with sql_conn() as conn:
        return pd.read_sql_query(draft_year_query, conn, params= (DraftYear,))

In [179]:
get_draft_results_by_year(2023)

,CollegeAthleteId,Name,Position,College,HighSchoolRank,NflTeam,DraftYear,HometownInfoLatitude,HometownInfoLongitude,Overall_Pick,NFLPreDraftGrade,ChangeInRating
0,4685720.0,Bryce Young,Quarterback,Alabama,0.9994,Carolina,2023,34.147645,-118.144478,1,96.0,-0.0394
1,4432577.0,C.J. Stroud,Quarterback,Ohio State,NaN,Houston,2023,34.103319,-117.575174,2,94.0,NaN
2,4685724.0,Will Anderson Jr.,Outside Linebacker,Alabama,0.9913,Houston,2023,33.387058,-84.282978,3,95.0,-0.0413
3,4429084.0,Anthony Richardson,Quarterback,Florida,0.9204,Indianapolis,2023,29.651968,-82.324985,4,91.0,-0.0104
4,4575431.0,Devon Witherspoon,Cornerback,Illinois,NaN,Seattle,2023,30.421309,-87.216915,5,93.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
254,4570410.0,Jalen Graham,Outside Linebacker,Purdue,0.8614,San Francisco,2023,42.331551,-83.046640,255,38.0,-0.4814
255,4879650.0,Grant DuBose,Wide Receiver,Charlotte,NaN,Green Bay,2023,32.366966,-86.300648,256,38.0,NaN
256,4242503.0,Alex Forsyth,Center,Oregon,0.8717,Denver,2023,45.365676,-122.612314,257,51.0,-0.3617
257,4360745.0,Kendall Williamson,Cornerback,Stanford,0.8854,Chicago,2023,33.857328,-84.019911,258,NaN,NaN


In [180]:
get_draft_results_by_year(2024).tail(70)

,CollegeAthleteId,Name,Position,College,HighSchoolRank,NflTeam,DraftYear,HometownInfoLatitude,HometownInfoLongitude,Overall_Pick,NFLPreDraftGrade,ChangeInRating
186,4428796.0,Casey Washington,Wide Receiver,Illinois,0.8323,Atlanta,2024,30.508235,-97.678893,187,30.0,-0.5323
187,4427479.0,Jamal Hill,Linebacker,Oregon,0.8755,Houston,2024,33.583166,-84.339368,188,NaN,NaN
188,4586333.0,Mekhi Wingo,Defensive Tackle,LSU,0.8701,Detroit,2024,38.626418,-90.199838,189,74.0,-0.1301
189,4430155.0,Dylan McMahon,Offensive Guard,NC State,0.8647,Philadelphia,2024,32.080926,-81.091177,190,56.0,-0.3047
190,4875912.0,Tejhaun Palmer,Wide Receiver,UAB,NaN,Arizona,2024,43.034993,-87.922497,191,30.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
251,4426479.0,Cornelius Johnson,Wide Receiver,Michigan,0.9294,Los Angeles,2024,41.026486,-73.628460,253,50.0,-0.4294
252,4373079.0,KT Leveston,Offensive Guard,Kansas State,0.8510,Los Angeles,2024,31.549333,-97.146670,254,54.0,-0.3110
253,4431433.0,Kalen King,Cornerback,Penn State,0.9131,Green Bay,2024,42.331551,-83.046640,255,40.0,-0.5131
254,4368033.0,Nick Gargiulo,Center,South Carolina,NaN,Denver,2024,41.269596,-73.775270,256,48.0,NaN


In [181]:
def inspect_null_drafts(DraftYear):
    null_draftyear_values = """
        SELECT *
        FROM Change_IN_Rating
        WHERE DraftYear = ? AND (Name IS NULL OR CollegeAthleteId IS NULL)
    """
    with sql_conn() as conn:
        return pd.read_sql_query(null_draftyear_values, conn, params=(DraftYear,))

In [182]:
inspect_null_drafts(2023)

,CollegeAthleteId,Name,Position,College,HighSchoolRank,NflTeam,DraftYear,HometownInfoLatitude,HometownInfoLongitude,Overall_Pick,NFLPreDraftGrade,ChangeInRating
